In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors

# Will plot in separate pop-up window
%matplotlib qt

In [2]:
# Read in GISTEMP data as dataframe
gistemp_file = 'gistemp_monthly.csv'
df_full = pd.read_csv(gistemp_file, header=1)
df_full = df_full.set_index('Year')

# Drop non-monthly columns
df = df_full.iloc[:, :12]

# Drop 2023 (incomplete)
df = df.head(len(df) - 1)

# Convert to floats
df = df.astype(float)

In [3]:
# Create color map based on IPCC visual style guide
# https://www.ipcc.ch/site/assets/uploads/2019/04/IPCC-visual-style-guide.pdf
# Temperature section, page 10

# Set RGB values (using 5 color scheme)
red = [202, 0, 32]
orange = [244, 165, 130]
white = [247, 247, 247]
light_blue = [146, 197, 222]
blue = [5, 113, 176]

# Set RGB values (using 11 color scheme)
red1 = [103, 0, 31]
red2 = [178, 24, 43]
red3 = [214, 96, 77]
red4 = [244, 165, 130]
red5 = [253, 219, 199]
white = [247, 247, 247]
blue5 = [209, 229, 240]
blue4 = [146, 197, 222]
blue3 = [67, 147, 195]
blue2 = [33, 102, 172]
blue1 = [5, 48, 97]

# Normalize values
color_list_5 = [blue, light_blue, white, orange, red]
color_list_11 = [blue1, blue2, blue3, blue4, blue5, white, red5, red4, red3, red2, red1]
color_tuples = []
for color in color_list_11:
    color_tuples.append([x/256 for x in color])

# Create cmap for later plotting
IPCC_cmap = matplotlib.colors.LinearSegmentedColormap.from_list('IPCC_RGB', color_tuples)

In [4]:
def monthly_timeseries(column='Jan'):
    
    # Initialize lists/counter, set values for colorbar and number of points
    temp_list = []
    year_list = []
    counter = 0
    num_points = len(df) - 1
    max_color = 1.25
    min_color = -max_color

    while True:

        # Gather values cumulatively
        temp = df[column].iloc[counter]
        year_list.append(int(df.index[counter]))
        temp_list.append(temp)

        # Line plot
        plt.rcParams["figure.figsize"] = (10, 10)
        plt.plot(year_list, temp_list, color='black')
        
        # Horizontal line for baseline temperature
        plt.axhline(y=0, color='black', linestyle='dashed')
        
        # Vertical line for anomaly baseline years
        plt.axvline(x=1951, color='black', linestyle='dashed')
        plt.axvline(x=1980, color='black', linestyle='dashed')

        # Set vertical bounds
        ymin = min(df[column]) * 1.1
        ymax = max(df[column]) * 1.1
        ymin = round(ymin, 1) - 0.05
        ymax = round(ymax, 1) + 0.05
        
        # Scatter plot     
        plt.scatter(year_list, temp_list, c=temp_list, cmap=IPCC_cmap)
        plt.clim(min_color, max_color)

        # Formatting
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Temperature Anomaly (\xb0C)', fontsize=14)
        plt.ylim(ymin, ymax)
        title = 'GISTEMP Global Temperature Anomaly'
        title += ' ('  + column +')'                    
        plt.title(title, fontsize=18)
        plt.text(x=1952, y=ymax*0.8, s='Time period \nused for \nbaseline\nanomaly')
        plt.draw()
        plt.grid()
        plt.colorbar(boundaries = np.arange(ymin, ymax, 0.05),
                     ticks=np.linspace(min_color, max_color, 11)).set_label('\xb0C', fontsize=14)
        
        # Brief pause
        plt.pause(0.05)

        # Clear plot to make room for next one
        if counter < num_points:
            plt.clf()
            
        # If last plot, stop plotting
        else:
            break

        counter += 1

In [8]:
monthly_timeseries('Oct')

In [6]:
# Similar to the above plot, but including every monthly data point
def overall_timeseries():

    # Get a list of all monthly values
    values = []
    dates = []

    # Create list of strings for each month
    month_list = []
    for i in range(12):
        month_num = str(i+1)
        if len(month_num) < 2:
            month_num = '0' + month_num
        month_list.append(month_num)

    # Collect datetime and values for each entry in dataframe
    for i in range(len(df)):
        year = df.iloc[i].name
        months = df.iloc[i].index
        val_i = df.iloc[i].values
        month_counter = 0
        for j in range(len(val_i)-1):
            values.append(val_i[j])
            date_str = str(year) + '-' + month_list[month_counter]
            dt = datetime.datetime.strptime(date_str, '%Y-%m')
            dates.append(dt)
            month_counter += 1

    # Initialize lists/counter, set values for colorbar and number of points
    temp_list = []
    year_list = []
    counter = 0
    num_points = len(values) - 1
    max_color = 1.25
    min_color = -max_color

    while True:

        # Gather values cumulatively
        temp_list.append(values[counter])
        year_list.append(dates[counter])

        # Line plot
        plt.rcParams["figure.figsize"] = (10, 10)
        plt.plot(year_list, temp_list, color='black')

        # Horizontal line for baseline temperature anomaly
        plt.axhline(y=0, color='black', linestyle='dashed')
        
        # Set vertical bounds
        ymin = min(df.min()) * 1.1
        ymax = max(df.max()) * 1.1
        ymin = round(ymin, 1) - 0.05
        ymax = round(ymax, 1) + 0.05
        
        # Scatter plot
        plt.scatter(year_list, temp_list, c=temp_list, cmap=IPCC_cmap)
        plt.clim(min_color, max_color)

        # Formatting
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Temperature Anomaly (\xb0C)', fontsize=14)
        ymin = min(values) * 1.1
        ymax = max(values) * 1.1
        plt.ylim(ymin, ymax)
        title = 'GISTEMP Global Temperature Anomaly'
        plt.title(title, fontsize=18)
        plt.draw()
        plt.grid()
        plt.colorbar(boundaries = np.arange(ymin, ymax, 0.05),
                     ticks=np.linspace(min_color, max_color, 11)).set_label('\xb0C', fontsize=14)

        # Very small pause
        plt.pause(10**(-10))

        # Clear plot to make room for next one
        if counter < num_points:
            plt.clf()
            
        # If last plot, stop plotting
        else:
            break

        counter += 1

In [7]:
overall_timeseries()